# Explore source measurement on a custom coadd

**Status:** Figured out one needed config override, but the new error seems unrelated to something the user can easily define. Need some help with "ValueError: Gen2 (cal_ref_cat) and Gen3 (ref_cat) reference catalogs are different.  These options must be kept in sync until Gen2 is retired."

In [1]:
import time
import numpy as np
import pandas

import lsst.geom
import lsst.afw.display as afwDisplay

from lsst.daf.butler import Butler

# lsst packages for executing pipeline tasks
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [2]:
config = "dp02"
collection = "u/melissagraham/coadd_recreation_nb"
outputRun = "u/melissagraham/coadd_recreation_nb/TestWindow1"
my_dataId = {'band': 'i', 'tract': 4431, 'patch': 17}

<br>

**Optional:** Get the coadd I made in draft_Create_Custom_Coadd.ipynb

In [3]:
# butler = Butler(config, collections=collection)
# my_coadd = butler.get('deepCoadd', dataId=my_dataId)

In [4]:
# my_coadd_inputs = butler.get("deepCoadd.coaddInputs", my_dataId)
# my_coadd_inputs.visits.asAstropy()

In [5]:
# del butler, my_coadd, my_coadd_inputs

<br>

**simpleButler**

In [6]:
simpleButler = Butler(config, collections=[collection], run=outputRun, writeable=True)

In [7]:
simpleButler.registry.getCollectionChain(collection)

CollectionSearch(('u/melissagraham/coadd_recreation_nb/TestWindow1', '2.2i/runs/DP0.2'))

In [8]:
### what kind of refcats are there
# for c in sorted(simpleButler.registry.queryCollections()):
#     if c.find('ref') > -1:
#         print(c)

<br>

**Set up the pipeline.**

In the DRP.yaml file, the source detection steps that come after assembleCoadd are:
 - healSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.HealSparsePropertyMapTask
 - consolidateHealSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.ConsolidateHealSparsePropertyMapTask
 - detection: lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask
 - mergeDetections: lsst.pipe.tasks.mergeDetections.MergeDetectionsTask
 - deblend: lsst.pipe.tasks.deblendCoaddSourcesPipeline.DeblendCoaddSourcesMultiTask
 - measure: lsst.pipe.tasks.multiBand.MeasureMergedCoaddSourcesTask
 - mergeMeasurements: lsst.pipe.tasks.mergeMeasurements.MergeMeasurementsTask
 - writeObjectTable: lsst.pipe.tasks.postprocess.WriteObjectTableTask
 
Not needed?
 - healSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.HealSparsePropertyMapTask
 - consolidateHealSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.ConsolidateHealSparsePropertyMapTask

Starting with `detection` and all the lists tasks after it.

In [9]:
yaml_file = '${PIPE_TASKS_DIR}/pipelines/DRP.yaml'
steps = 'detection,mergeDetections,deblend,measure,mergeMeasurements,writeObjectTable'
my_uri = yaml_file + '#' + steps

print(my_uri)

${PIPE_TASKS_DIR}/pipelines/DRP.yaml#detection,mergeDetections,deblend,measure,mergeMeasurements,writeObjectTable


In [10]:
measureSourcesPipeline = Pipeline.from_uri(my_uri)

<br>

**query string**

In [11]:
queryString = "tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'"
print(queryString)

tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'


<br>

**Configurations**

`'mergeDetections', 'priorityList', 'i'` -- from an error when attempting to make spe I figured this one out

In [12]:
measureSourcesPipeline.addConfigOverride('mergeDetections', 'priorityList', 'i')

In [13]:
# measureSourcesPipeline.addConfigOverride('measure', 'ref_dataset_name', 'None')

<br>

>**Help needed:** I've done a bunch of poking around but I can't figure out how to change the refObjLoader value of 'ref_dataset_name', or why anything Gen2 matters? This doesn't seem to be a configuration that can be overrided.

> Can see the part of the code here:
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/multiBand.py#L824

In [14]:
%%time
spe = SimplePipelineExecutor.from_pipeline(measureSourcesPipeline, where=queryString, butler=simpleButler)

ValueError: Gen2 (cal_ref_cat) and Gen3 (ref_cat) reference catalogs are different.  These options must be kept in sync until Gen2 is retired.

In [15]:
del simpleButler

Can we look at the measure config and see what might have been used for the general processing?

In [ ]:
butler = Butler(config, collections='2.2i/runs/DP0.2')

Don't use a dataId? <br>
`DatasetType('measure_config', {}, Config)`

In [ ]:
### 
# test = butler.get("measure_config", dataId=my_dataId)
test = butler.get("measure_config")

In [ ]:
for key,value in test.items():
    print(key,value)

In [ ]:
del butler

<br><br><br><br><br><br><br>


# earlier attempts. ignore below


## 2. Characterize Image Task

> **Help Question:** Is it necessary to run CharacterizeImageTask on the coadd? It seems the image already has psf, is already characterized?

In [ ]:
my_coadd_bbox = butler.get("deepCoadd.bbox", dataId=my_dataId)
x_val = my_coadd_bbox.beginX + 200
y_val = my_coadd_bbox.beginY + 200
point = lsst.geom.Point2D(x_val, y_val)

psf = my_coadd.getPsf()
psfShape = psf.computeShape(point)
sigma = psfShape.getDeterminantRadius()
pixelScale = my_coadd.getWcs().getPixelScale().asArcseconds()
print('psf fwhm = {:.2f} arcsec at x = {:.0f}, y = {:.0f}'.format(sigma*pixelScale*2.355,x_val,y_val))

If it's necessary, do like this?

In [ ]:
# from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

# ci_config = CharacterizeImageTask.ConfigClass()
# ci_config.psfIterations = 1
# charImageTask = CharacterizeImageTask(config=ci_config)

# result = charImageTask.run(my_coadd)

## 3. Configure Source Detection, Deblend, and Measurement Tasks

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
tab = afwTable.SourceTable.make(schema)

In [ ]:
schema

In [ ]:
sd_config = SourceDetectionTask.ConfigClass()
sd_config.thresholdValue = 5
sd_config.thresholdType = "stdev"

sourceDetectionTask = SourceDetectionTask(schema=schema, config=sd_config)

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

In [ ]:
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
sm_config = SingleFrameMeasurementTask.ConfigClass()

sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=sm_config,
                                                   algMetadata=algMetadata)

## 4. Run Source Detection Task

In [ ]:
%%time
result = sourceDetectionTask.run(tab, my_coadd)

In [ ]:
sources = result.sources

In [ ]:
print(len(sources), result.numPosPeaks)

In [ ]:
sources.asAstropy()

In [ ]:
tx = np.where( np.isfinite( sources['coord_ra'] ) )[0]
print(len(tx))
del tx

## 5. Run Source Deblending

https://pipelines.lsst.io/modules/lsst.meas.deblender/index.html

https://pipelines.lsst.io/modules/lsst.meas.deblender/tasks/lsst.meas.deblender.SourceDeblendTask.html

> **Help Question:** `sourceDeblendTask.run` exists with an assertion error about the schemas even when we make sure the input schema matches before we start. Not sure how to fix.

SourceDeblendTask modifies the source catalog in-place.

In [ ]:
assert sources.getSchema() == schema

Attempt to fix that by redefining schema and sourceDeblendTask.

In [ ]:
schema = sources.getSchema()

In [ ]:
assert sources.getSchema() == schema

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

Does not seem to help, below we still get the error:
> `--> 275 assert sources.getSchema() == self.schema`

In [ ]:
%%time
sourceDeblendTask.run(my_coadd, sources)

In [ ]:
print(len(sources))

## 6. Run Source Measurement

https://pipelines.lsst.io/modules/lsst.meas.base/tasks/lsst.meas.base.sfm.SingleFrameMeasurementTask.html

In [ ]:
%%time
sourceMeasurementTask.run(measCat=sources, exposure=my_coadd)